Below is an implementation of an autoencoder written in PyTorch. We apply it to the MNIST dataset.

In [1]:
# o

# from google.colab import drive
# drive.mount('/content/drive')
# base_dir = '/content/drive/My Drive/gary_class_project/'
# save_dir = base_dir + 'results'

In [2]:
import os
base_dir = os.path.abspath(os.getcwd())
save_dir = os.path.join(base_dir,'results')


In [3]:
# !pip install wandb -q
# !pip install neurora -q

#wandb api key 18a861e71f78135d23eb672c08922edbfcb8d364

In [4]:
import torch; torch.manual_seed(0)
import wandb
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
from tqdm import tqdm
import random
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from neurora.rdm_corr import rdm_correlation_spearman

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Main code to run code on various models while varying the amount of training data

## Setting up the model

In [6]:
class Encoder(nn.Module):
    def __init__(self, encoded_space_dim, num_classes):
        super().__init__()
        ""
        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(8, 16, 3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 3, stride=2, padding=0),
            nn.ReLU(True)
        )
        
        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        self.encoder_labels_lin = nn.Linear(num_classes, num_classes//2)
        ### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(3 * 3 * 32 + num_classes//2, 128),
            nn.ReLU(True),
            nn.Linear(128, encoded_space_dim)
        )
        
        
    def forward(self, x, y):
        img_features = self.encoder_cnn(x)
        img_features = self.flatten(img_features)
        label_features = self.encoder_labels_lin(y)
        combined = torch.cat((img_features, label_features), dim = -1)
        out = self.encoder_lin(combined)
        return out

class Decoder(nn.Module):
    
    def __init__(self, encoded_space_dim, num_classes):
        super().__init__()
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 3 * 3 * 32 + (num_classes)//2),
            nn.ReLU(True)
        )
        self.decoder_labels_lin = nn.Linear((num_classes)//2, num_classes)
        
        self.unflatten = nn.Unflatten(dim=1, 
        unflattened_size=(32, 3, 3))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, 
            stride=2, output_padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2, 
            padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=2, 
            padding=1, output_padding=1)
        )
        
    def forward(self, x):
        x = self.decoder_lin(x)
        img_features = x[:, :-(num_classes//2)]
        label_features = x[:, -(num_classes//2):]
        img_features = self.unflatten(img_features)
        img_features = self.decoder_conv(img_features)
        img = torch.sigmoid(img_features)
        label = self.decoder_labels_lin(label_features)
        return img, label

In [7]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()
        # self.gamma = nn.Parameter(torch.tensor([.5]))
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, img, pred_img, label, pred_label):
        mse_loss_img = ((img - pred_img)**2).sum()
        mse_loss_label = self.cross_entropy(pred_label, label.float())
        # loss = mse_loss_img * torch.sigmoid(self.gamma) + \
              # mse_loss_label * (1 - torch.sigmoid(self.gamma))
        loss = mse_loss_img + mse_loss_label#*torch.sigmoid(self.gamma)
        return mse_loss_img, mse_loss_label, loss

class Autoencoder(nn.Module):
    def __init__(self, latent_dims, num_classes):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims, num_classes)
        self.decoder = Decoder(latent_dims, num_classes)
        self.custom_loss = CustomLoss()
        self.num_classes = num_classes
    
    def forward(self, x, y):
        z = self.encoder(x, y)
        return self.decoder(z)

    def test_epoch(self, test_data):
    # Set evaluation mode for encoder and decoder
        self.eval()
        with torch.no_grad(): # No need to track the gradients
            # Define the lists to store the outputs for each batch
            test_img_loss = []
            test_label_loss = []
            total_test_loss = []
            for image_batch, label_batch in test_data:
                total = 0
                correct = 0
                # Move tensor to the proper device
                image_batch = image_batch.to(device)
                label_batch = F.one_hot(label_batch, num_classes=self.num_classes)
                label_batch = label_batch.to(device)
                pred_img, pred_label = self.forward(image_batch, label_batch.float()) 
                # Append the network output and the original image to the lists
                img_loss, label_loss, total_loss = self.custom_loss(image_batch,
                                                                pred_img, 
                                                                label_batch, 
                                                                pred_label)
                total += label_batch.size(0)
                correct += (torch.argmax(pred_label, dim = 1) == torch.argmax(label_batch, dim = 1)).sum().item()
                test_img_loss.append(img_loss.item())
                test_label_loss.append(label_loss.item())
                total_test_loss.append(total_loss.item())
        test_img_loss = sum(test_img_loss)/len(test_img_loss)
        test_label_loss = sum(test_label_loss)/len(test_label_loss)
        total_test_loss = sum(total_test_loss)/len(total_test_loss)
        test_accuracy = correct/total
        return test_img_loss, test_label_loss, total_test_loss, test_accuracy

    def train_epoch(self, train_data, optimizer, training_label_ratio):
        self.train()
        torch.manual_seed(0)
        train_img_loss = []
        train_label_loss = []
        train_loss = []
        correct = 0
        total = 0
        for image_batch, label_batch in train_data:
            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            num_training_examples = label_batch.shape[0]
            num_non_label_training_examples = num_training_examples*(1-training_label_ratio)
            non_label_training_idx = random.sample(range(num_training_examples),int(num_non_label_training_examples))
            label_batch[[non_label_training_idx]] = self.num_classes - 1
            label_batch = F.one_hot(label_batch, num_classes=self.num_classes)
            label_batch = label_batch.to(device)
            optimizer.zero_grad()
            pred_img, pred_label = self.forward(image_batch, label_batch.float()) 
            # Append the network output and the original image to the lists
            img_loss, label_loss, total_loss = self.custom_loss(image_batch,
                                                            pred_img, 
                                                            label_batch, 
                                                            pred_label)
            total_loss.backward()
            optimizer.step()
            train_img_loss.append(img_loss.item())
            train_label_loss.append(label_loss.item())
            train_loss.append(total_loss.item())
            total += label_batch.size(0)
            correct += (torch.argmax(pred_label, dim = 1) == torch.argmax(label_batch, dim = 1)).sum().item()
        train_img_loss = sum(train_img_loss)/len(train_img_loss)
        train_label_loss = sum(train_label_loss)/len(train_label_loss)
        train_loss = sum(train_loss)/len(train_loss)
        train_accuracy = correct/total
        return train_img_loss, train_label_loss, train_loss, train_accuracy

    def training_loop(self, train_data, test_data,training_label_ratio,
                      epochs, optimizer):
        train_losses = []
        val_losses = []
        train_img_losses = []
        val_img_losses = []
        train_label_losses = []
        val_label_losses = []
        train_accuracies = []
        val_accuracies = []
        for epoch in tqdm(range(epochs)):
          train_img_loss, train_label_loss, train_loss, train_accuracy =self.train_epoch(train_data, optimizer, 
                                             training_label_ratio)
          val_img_loss, val_label_loss, val_loss, val_accuracy = self.test_epoch(test_data)
          train_losses.append(train_loss)
          val_losses.append(val_loss)
          train_img_losses.append(train_img_loss)
          val_img_losses.append(val_img_loss)
          train_label_losses.append(train_label_loss)
          val_label_losses.append(val_label_loss)
          train_accuracies.append(train_accuracy)
          val_accuracies.append(val_accuracy)
          wandb.log({"train_img_loss": train_img_loss, 
            "train_label_loss":train_label_loss, 
            "val_img_loss":val_img_loss, 
            "val_label_loss":val_label_loss, 
            "train_losses":train_loss, 
            "val_losses":val_loss, 
            "train_accuracy":train_accuracy, 
            "val_accuracy":val_accuracy})

        return train_img_losses, train_label_losses, val_img_losses, val_label_losses ,train_losses, val_losses, train_accuracies, val_accuracies

def plot_latent_with_label(autoencoder, batch_size, data, random_labels,
                           num_classes, num_batches=100):
    if not random_labels:
        for i, (x, y) in enumerate(data):
            x = x.to(device) # GPU
            y_one_hot = F.one_hot(y, num_classes)
            y_one_hot = y_one_hot.to(device)
            z = autoencoder.encoder(x, y_one_hot.float())
            z = z.to('cpu').detach().numpy()
            plt.scatter(z[:, 0], z[:, 1], c=y, cmap='tab10')
            if i > num_batches:
                plt.colorbar()
                break
        wandb.log({"Latent : Real labels in input": plt})
    else:
        for i, (x, y) in enumerate(data):
            x = x.to(device) # GPU
            y_rand = torch.zeros((x.size(0), num_classes))
            y_rand = y_rand.to(device)
            z = autoencoder.encoder(x, y_rand.float())
            z = z.to('cpu').detach().numpy()
            plt.scatter(z[:, 0], z[:, 1], c=y, cmap='tab10')
            if i > num_batches:
                plt.colorbar()
                break
        wandb.log({"Latent : Random labels in input": plt})
    return min(z[:, 0]), max(z[:, 0]), min(z[:, 1]), max(z[:, 1])


def plot_reconstructed_with_labels(autoencoder,random_labels, r0,
                                   r1, n=24):
    w = 28
    img = np.zeros((n*w, n*w))
    for i, y in enumerate(np.linspace(*r1, n)):
        for j, x in enumerate(np.linspace(*r0, n)):
            z = torch.Tensor([[x, y]]).to(device)
            x_hat, label = autoencoder.decoder(z)
            x_hat = x_hat.reshape(28, 28).to('cpu').detach().numpy()
            img[(n-1-i)*w:(n-1-i+1)*w, j*w:(j+1)*w] = x_hat
    plt.imshow(img, extent=[*r0, *r1])
    if not random_labels:
      wandb.log({"Reconstruction : Real labels in input": plt})
    else:
      wandb.log({"Reconstruction : Random labels in input": plt})
    
    

## Training multiple networks

In [8]:
def wandb_init(epochs, lr, label_ratio, batch_size, model_number):
  wandb.init(project="ConceptualAlignmentLangugae", entity="psych-711")
  wandb.config = {
    "learning_rate": lr,
    "epochs": epochs,
    "batch_size": batch_size, 
    "label_ratio":label_ratio, 
    "model_number": model_number 
  }
  wandb.run.name = '{}_{}'.format(label_ratio, model_number)
  wandb.run.save()
     

In [9]:
def main_code(save_dir, num_models, epochs, num_classes, batch_size,
             lr, latent_dims):
  if os.path.isdir(save_dir):
    pass
  else:
    os.mkdir(save_dir)

  for i in tqdm(range(0, 11)):
    torch.manual_seed(0)
    label_ratio = i/10
    for model in range(num_models):
      wandb_init(epochs, lr, label_ratio, batch_size, model)
      train_data = torch.utils.data.DataLoader( 
        torchvision.datasets.MNIST('./data', 
               transform=torchvision.transforms.ToTensor(), train = True, 
               download=True),
        batch_size=batch_size,
        shuffle=True)
      train_data, val_data = torch.utils.data.random_split(train_data.dataset, 
                                                           [50000, 10000])
      train_data = torch.utils.data.DataLoader(train_data, 
                                               batch_size=batch_size,
                                            shuffle=True)
      val_data = torch.utils.data.DataLoader(val_data, 
                                               batch_size=batch_size,
                                            shuffle=True)
      test_data = torch.utils.data.DataLoader( 
              torchvision.datasets.MNIST('./data', 
                    transform=torchvision.transforms.ToTensor(), train = False, 
                    download=True),
              batch_size=batch_size,
              shuffle=True)
      autoencoder = Autoencoder(latent_dims, num_classes).to(device) # GPU
      optimizer = torch.optim.Adam(autoencoder.parameters(), lr=lr, weight_decay=1e-05)
      train_img_loss, train_label_loss, val_img_loss, \
      val_label_loss ,train_losses, val_losses,  train_accuracy, \
      val_accuracy= autoencoder.training_loop(train_data = train_data,
                                                          test_data = test_data,
                                                          epochs = epochs,
                                                          optimizer = optimizer, 
                                                          training_label_ratio = label_ratio)
      # figure out which data we want to plot

      min_x, max_x, min_y, max_y = plot_latent_with_label(autoencoder, 
                                                          batch_size, 
                                                          data=val_data, 
                                                          random_labels = False,
                                                          num_classes = num_classes,
                                                          num_batches=100)
      plt.clf()
      plot_reconstructed_with_labels(autoencoder = autoencoder, 
                                     r0=(min_x, max_x),
                                    r1=(min_y, max_y), 
                                     n=24, random_labels = False)
      plt.clf()
      min_x, max_x, min_y, max_y = plot_latent_with_label(autoencoder, 
                                                          batch_size, 
                                                          data=val_data, 
                                                          random_labels = True,
                                                          num_classes = num_classes,
                                                          num_batches=100)
      plt.clf()
      plot_reconstructed_with_labels(autoencoder = autoencoder, 
                                     r0=(min_x, max_x),
                                    r1=(min_y, max_y), 
                                     n=24, random_labels = True)
      plt.clf()
      
      # wandb.log({"train_img_loss": train_img_loss, 
      #            "train_label_loss":train_label_loss, 
      #            "val_img_loss":val_img_loss, 
      #            "val_label_loss":val_label_loss, 
      #            "train_losses":train_losses, 
      #            "val_losses":val_losses, 
      #            "train_accuracy":train_accuracy, 
      #            "val_accuracy":val_accuracy})
      torch.save(autoencoder.state_dict(), save_dir+'_{}_{}'.format(label_ratio, 
                                                                    model))
      
      


## Main program code

In [10]:
import plotly

In [11]:
num_classes = 11 # Number of unique class labels in the dataset
latent_dims = 2
epochs = 1
lr = 0.001
num_models = 2
batch_size = 512
save_dir = base_dir + 'results/'
main_code(save_dir, num_models, epochs, num_classes, batch_size,
             lr, latent_dims)

  0%|          | 0/11 [00:00<?, ?it/s]wandb: Currently logged in as: psych-711 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
100%|██████████| 1/1 [00:10<00:00, 10.69s/it]
/Users/kushin/.local/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.



train_accuracy,▁
train_img_loss,▁
train_label_loss,▁
train_losses,▁
val_accuracy,▁
val_img_loss,▁
val_label_loss,▁
val_losses,▁
train_accuracy,1.0
train_img_loss,72703.18614
train_label_loss,1.33061


100%|██████████| 1/1 [00:10<00:00, 10.83s/it]
/Users/kushin/.local/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.

  9%|▉         | 1/11 [00:49<08:15, 49.56s/it]

train_accuracy,▁
train_img_loss,▁
train_label_loss,▁
train_losses,▁
val_accuracy,▁
val_img_loss,▁
val_label_loss,▁
val_losses,▁
train_accuracy,0.58318
train_img_loss,80301.14565
train_label_loss,1.89472


...

100%|██████████| 1/1 [00:10<00:00, 10.45s/it]
/Users/kushin/.local/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.



train_accuracy,▁
train_img_loss,▁
train_label_loss,▁
train_losses,▁
val_accuracy,▁
val_img_loss,▁
val_label_loss,▁
val_losses,▁
train_accuracy,0.0222
train_img_loss,72659.83566
train_label_loss,2.42466


100%|██████████| 1/1 [00:10<00:00, 10.50s/it]
/Users/kushin/.local/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.

100%|██████████| 11/11 [12:16<00:00, 66.97s/it]


<Figure size 1200x800 with 0 Axes>

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
Exception in thread Stat

# Code to compute RDMs and their correlations

## Setting up the model

In [5]:
class Encoder(nn.Module):
    def __init__(self, encoded_space_dim, num_classes):
        super().__init__()
        
        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(8, 16, 3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 3, stride=2, padding=0),
            nn.ReLU(True)
        )
        
        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        self.encoder_labels_lin = nn.Linear(num_classes, num_classes//2)
        ### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(3 * 3 * 32 + num_classes//2, 128),
            nn.ReLU(True),
            nn.Linear(128, encoded_space_dim)
        )
        
        
    def forward(self, x, y):
        img_features = self.encoder_cnn(x)
        img_features = self.flatten(img_features)
        label_features = self.encoder_labels_lin(y)
        combined = torch.cat((img_features, label_features), dim = -1)
        out = self.encoder_lin(combined)
        return out

class Decoder(nn.Module):
    
    def __init__(self, encoded_space_dim, num_classes):
        super().__init__()
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 3 * 3 * 32 + (num_classes)//2),
            nn.ReLU(True)
        )
        self.decoder_labels_lin = nn.Linear((num_classes)//2, num_classes)
        
        self.unflatten = nn.Unflatten(dim=1, 
        unflattened_size=(32, 3, 3))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, 
            stride=2, output_padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2, 
            padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=2, 
            padding=1, output_padding=1)
        )
        
    def forward(self, x):
        x = self.decoder_lin(x)
        img_features = x[:, :-(num_classes//2)]
        label_features = x[:, -(num_classes//2):]
        img_features = self.unflatten(img_features)
        img_features = self.decoder_conv(img_features)
        img = torch.sigmoid(img_features)
        label = self.decoder_labels_lin(label_features)
        return img, label

In [6]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()
        # self.gamma = nn.Parameter(torch.tensor([.5]))
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, img, pred_img, label, pred_label):
        mse_loss_img = ((img - pred_img)**2).sum()
        mse_loss_label = self.cross_entropy(pred_label, label.float())
        # loss = mse_loss_img * torch.sigmoid(self.gamma) + \
              # mse_loss_label * (1 - torch.sigmoid(self.gamma))
        loss = mse_loss_img + mse_loss_label#*torch.sigmoid(self.gamma)
        return mse_loss_img, mse_loss_label, loss

class Autoencoder(nn.Module):
    def __init__(self, latent_dims, num_classes):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims, num_classes)
        self.decoder = Decoder(latent_dims, num_classes)
        self.custom_loss = CustomLoss()
        self.num_classes = num_classes
    
    def forward(self, x, y):
        z = self.encoder(x, y)
        return self.decoder(z)

    def test_epoch(self, test_data):
    # Set evaluation mode for encoder and decoder
        self.eval()
        with torch.no_grad(): # No need to track the gradients
            # Define the lists to store the outputs for each batch
            test_img_loss = []
            test_label_loss = []
            total_test_loss = []
            for image_batch, label_batch in test_data:
                total = 0
                correct = 0
                # Move tensor to the proper device
                image_batch = image_batch.to(device)
                label_batch = F.one_hot(label_batch, num_classes=self.num_classes)
                label_batch = label_batch.to(device)
                pred_img, pred_label = self.forward(image_batch, label_batch.float()) 
                # Append the network output and the original image to the lists
                img_loss, label_loss, total_loss = self.custom_loss(image_batch,
                                                                pred_img, 
                                                                label_batch, 
                                                                pred_label)
                total += label_batch.size(0)
                correct += (torch.argmax(pred_label, dim = 1) == torch.argmax(label_batch, dim = 1)).sum().item()
                test_img_loss.append(img_loss.item())
                test_label_loss.append(label_loss.item())
                total_test_loss.append(total_loss.item())
        test_img_loss = sum(test_img_loss)/len(test_img_loss)
        test_label_loss = sum(test_label_loss)/len(test_label_loss)
        total_test_loss = sum(total_test_loss)/len(total_test_loss)
        test_accuracy = correct/total
        return test_img_loss, test_label_loss, total_test_loss, test_accuracy

    def train_epoch(self, train_data, optimizer, training_label_ratio):
        self.train()
        torch.manual_seed(0)
        train_img_loss = []
        train_label_loss = []
        train_loss = []
        correct = 0
        total = 0
        for image_batch, label_batch in train_data:
            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            num_training_examples = label_batch.shape[0]
            num_non_label_training_examples = num_training_examples*(1-training_label_ratio)
            non_label_training_idx = random.sample(range(num_training_examples),int(num_non_label_training_examples))
            label_batch[[non_label_training_idx]] = self.num_classes - 1
            label_batch = F.one_hot(label_batch, num_classes=self.num_classes)
            label_batch = label_batch.to(device)
            optimizer.zero_grad()
            pred_img, pred_label = self.forward(image_batch, label_batch.float()) 
            # Append the network output and the original image to the lists
            img_loss, label_loss, total_loss = self.custom_loss(image_batch,
                                                            pred_img, 
                                                            label_batch, 
                                                            pred_label)
            total_loss.backward()
            optimizer.step()
            train_img_loss.append(img_loss.item())
            train_label_loss.append(label_loss.item())
            train_loss.append(total_loss.item())
            total += label_batch.size(0)
            correct += (torch.argmax(pred_label, dim = 1) == torch.argmax(label_batch, dim = 1)).sum().item()
        train_img_loss = sum(train_img_loss)/len(train_img_loss)
        train_label_loss = sum(train_label_loss)/len(train_label_loss)
        train_loss = sum(train_loss)/len(train_loss)
        train_accuracy = correct/total
        return train_img_loss, train_label_loss, train_loss, train_accuracy

    def training_loop(self, train_data, test_data,training_label_ratio,
                      epochs, optimizer):
        train_losses = []
        val_losses = []
        train_img_losses = []
        val_img_losses = []
        train_label_losses = []
        val_label_losses = []
        train_accuracies = []
        val_accuracies = []
        for epoch in tqdm(range(epochs)):
          train_img_loss, train_label_loss, train_loss, train_accuracy =self.train_epoch(train_data, optimizer, 
                                             training_label_ratio)
          val_img_loss, val_label_loss, val_loss, val_accuracy = self.test_epoch(test_data)
          train_losses.append(train_loss)
          val_losses.append(val_loss)
          train_img_losses.append(train_img_loss)
          val_img_losses.append(val_img_loss)
          train_label_losses.append(train_label_loss)
          val_label_losses.append(val_label_loss)
          train_accuracies.append(train_accuracy)
          val_accuracies.append(val_accuracy)
          wandb.log({"train_img_loss": train_img_loss, 
            "train_label_loss":train_label_loss, 
            "val_img_loss":val_img_loss, 
            "val_label_loss":val_label_loss, 
            "train_losses":train_loss, 
            "val_losses":val_loss, 
            "train_accuracy":train_accuracy, 
            "val_accuracy":val_accuracy})

        return train_img_losses, train_label_losses, val_img_losses, val_label_losses ,train_losses, val_losses, train_accuracies, val_accuracies

def plot_latent_with_label(autoencoder, batch_size, data, random_labels,
                           num_classes, num_batches=100):
    if not random_labels:
        for i, (x, y) in enumerate(data):
            x = x.to(device) # GPU
            y_one_hot = F.one_hot(y, num_classes)
            y_one_hot = y_one_hot.to(device)
            z = autoencoder.encoder(x, y_one_hot.float())
            z = z.to('cpu').detach().numpy()
            plt.scatter(z[:, 0], z[:, 1], c=y, cmap='tab10')
            if i > num_batches:
                plt.colorbar()
                break
        wandb.log({"Latent : Real labels in input": plt})
    else:
        for i, (x, y) in enumerate(data):
            x = x.to(device) # GPU
            y_rand = torch.zeros((x.size(0), num_classes))
            y_rand = y_rand.to(device)
            z = autoencoder.encoder(x, y_rand.float())
            z = z.to('cpu').detach().numpy()
            plt.scatter(z[:, 0], z[:, 1], c=y, cmap='tab10')
            if i > num_batches:
                plt.colorbar()
                break
        wandb.log({"Latent : Random labels in input": plt})
    return min(z[:, 0]), max(z[:, 0]), min(z[:, 1]), max(z[:, 1])


def plot_reconstructed_with_labels(autoencoder,random_labels, r0,
                                   r1, n=24):
    w = 28
    img = np.zeros((n*w, n*w))
    for i, y in enumerate(np.linspace(*r1, n)):
        for j, x in enumerate(np.linspace(*r0, n)):
            z = torch.Tensor([[x, y]]).to(device)
            x_hat, label = autoencoder.decoder(z)
            x_hat = x_hat.reshape(28, 28).to('cpu').detach().numpy()
            img[(n-1-i)*w:(n-1-i+1)*w, j*w:(j+1)*w] = x_hat
    plt.imshow(img, extent=[*r0, *r1])
    if not random_labels:
      wandb.log({"Reconstruction : Real labels in input": plt})
    else:
      wandb.log({"Reconstruction : Random labels in input": plt})
    
    

## Helper functions

In [11]:
def custom_torch_RSM_fct(features):
  """
  Custom function to calculate representational similarity matrix (RSM) of a feature
  matrix using pairwise cosine similarity.

  Args:
    features: 2D torch.Tensor
      Feature matrix of size (nbr items x nbr features)

  Returns:
    rsm: 2D torch.Tensor
      Similarity matrix of size (nbr items x nbr items)
  """
  features = torch.from_numpy(features)
  num_items, num_features = features.shape

  rsm = torch.nn.functional.cosine_similarity(
      features.reshape(1, num_items, num_features),
      features.reshape(num_items, 1, num_features),
      dim=2
      )

  if not rsm.shape == (num_items, num_items):
    raise ValueError(
        f"RSM should be of shape ({num_items}, {num_items})"
        )
  return rsm

In [12]:
def wandb_rsm_init():
  wandb.init(project="ConceptualAlignmentLangugae", entity="psych-711")
  wandb.run.name = 'rsms'
def compute_rsms(save_dir, batch_size, latent_dims, num_classes, test_data):
  '''
  Computes RSMs of all the models
  '''
  all_rsms = torch.rand((len(os.listdir(save_dir)), 10000 ,10000))
  print(all_rsms.shape)
  model_names = []
  print('Computing RSMs')
  for i, model in tqdm(enumerate(os.listdir(save_dir))):
    model_names.append(model)
    autoencoder = Autoencoder(latent_dims, num_classes).to(device) # GPU
    autoencoder.eval()
    autoencoder.load_state_dict(torch.load(os.path.join(save_dir, model)))
    autoencoder.eval()
    #initialise arrays
    latent_representations = np.asarray([[0, 0]])
    labels = np.asarray([0])
    for image_batch, label_batch in test_data:
        image_batch = image_batch.to(device)
        label_batch_one_hot = F.one_hot(label_batch, num_classes=num_classes)
        label_batch_one_hot = label_batch_one_hot.to(device)
        z = autoencoder.encoder.forward(image_batch, label_batch_one_hot.float()) 
        latent_representations = np.vstack((latent_representations, z.cpu().detach()))
        labels = np.concatenate((labels, label_batch.float().cpu().detach()))
    labels = labels[1:]    
    latent_representations = latent_representations[1:]
    sorted_idx = np.array([], dtype = np.int8)

    for label_class in range(num_classes):
      sorted_idx = np.concatenate((sorted_idx, np.where(labels==label_class)[0]))
    latent_representations = latent_representations[sorted_idx]
    rsm = custom_torch_RSM_fct(latent_representations)
    all_rsms[i] = rsm
    plt.imshow(rsm)
    plt.colorbar()
    wandb.log({"RSM : {}".format(model): plt})
    plt.clf()
    print('Done with RSMs')
  return all_rsms, model_names


def fill_upper_triangular(matrix):
  '''
  Fills in the upper trinagular matrix for a symmetric matrix which has
  lower traingular matrix filled iin
  '''
  items, features = matrix.shape
  assert(items==features)
  for i in range(items):
    for j in range(i+1, items):
      matrix[i][j] = matrix[j][i]
  return matrix

def log_rsm(matrix, var_name):
  plt.imshow(matrix)
  plt.colorbar()
  wandb.log({var_name: plt})
  plt.clf()
  

def compute_rsa(all_rsms, model_names):
  '''
  Computes pairwise RSA (and significance) for all the models given their rsms
  '''
  rsa_corr = torch.zeros([len(model_names), len(model_names)])
  rsa_pvalue = torch.zeros([len(model_names), len(model_names)])

  # Computes the lower triangular matrix irst 
  print('Computing RSA)')
  for i in tqdm(range(len(model_names))):
    for j in range(i +1):
      rsa_corr[i][j],rsa_pvalue[i][j]= rdm_correlation_spearman(all_rsms[i], 
                                                                all_rsms[j], 
                                                                rescale=False,
                                                                permutation=False, iter=1000)
  # Fills in the upper triangular matrix
  print('Done with RSA')
  rsa_corr = fill_upper_triangular(rsa_corr)
  rsa_pvalue = fill_upper_triangular(rsa_pvalue)
  log_rsm(rsa_corr, "RSA corr")
  log_rsm(rsa_pvalue, "RSA pvalue")
  return rsa_corr, rsa_pvalue


def main_analysis_loop(save_dir, batch_size, latent_dims, num_classes, 
                       test_data):
  '''
  Takes in all the models in a saved dir and computes pairwise rsa
  '''
  wandb_rsm_init()
  all_rsms, model_names = compute_rsms(save_dir, batch_size, latent_dims, 
                                       num_classes, test_data)
  rsa_corr, rsa_pvalue = compute_rsa(all_rsms, model_names)
  rsa_corr, rsa_pvalue = fill_upper_triangular(rsa_corr), fill_upper_triangular(rsa_pvalue)


## Run RSM analysis

In [13]:
num_classes = 11 # Number of unique class labels in the dataset
latent_dims = 2
batch_size = 1024

In [ ]:
test_data = torch.utils.data.DataLoader( 
      torchvision.datasets.MNIST('./data', 
            transform=torchvision.transforms.ToTensor(),
            download=True, train = False),
      batch_size=batch_size,
      shuffle=False)
main_analysis_loop(save_dir, batch_size, latent_dims, num_classes, 
                       test_data)

torch.Size([22, 10000, 10000])
Computing RSMs


1it [00:09,  9.97s/it]

Done with RSMs


2it [00:20, 10.19s/it]

Done with RSMs


...

21it [03:30,  9.98s/it]

Done with RSMs


22it [03:41, 10.06s/it]


Done with RSMs
Computing RSA)


  0%|          | 0/22 [00:00<?, ?it/s]